In [1]:
import os
# from langchain_community.llms import QianfanLLMEndpoint,Tongyi
# from langchain_community.chat_models import ChatZhipuAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import psycopg2
import re

In [2]:
# 百度千帆
# api = "xxxxxxxxx"
# sk = "xxxxxx"
# 
# llm = QianfanLLMEndpoint(model="ERNIE-4.0-8K",qianfan_ak=api,qianfan_sk=sk)

In [3]:
# 千问
# os.environ['DASHSCOPE_API_KEY'] = 'sk-cxxxxxx'
# 
# llm = Tongyi(model_name="qwen-max")

In [4]:
# 智谱
# api_key = "xxxxxx"

# os.environ["ZHIPUAI_API_KEY"] = "xxxxxxx"
# llm = ChatZhipuAI(
#     model="glm-4",
#     temperature=0.5,
# )

In [5]:
# deepseek
llm = ChatOpenAI(
    model = 'deepseek-chat',
    openai_api_key='sk-cxxxxxxx5', 
    openai_api_base='https://api.deepseek.com/v1'
)

In [6]:
input_text = "用50个字左右阐述，生命的意义在于"
response = llm.invoke(input_text)
print(response.content)

生命的意义在于追求成长、体验爱与奉献，创造价值，超越自我，与他人和世界建立深刻连接，实现内心的平静与满足。


In [7]:
# 数据库连接参数
dbname = "xxxx"
user = "xxxx"
password = "xxxx"
host = "xxxx"  # 或者是你 Docker 容器的 IP 地址，如果你在不同的机器上
port = "xxxx"

# 连接字符串
conn_string = f"host={host} dbname={dbname} user={user} password={password} port={port}"
# 连接到数据库
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [8]:
# 获取表名
def get_table_name(cur):
    # 执行 SQL 查询以获取所有表的列表
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    
    # 获取查询结果
    tables = cur.fetchall()
    tname = []
    # 打印表名
    for table in tables:
        tname.append(table[0])
    
    return tname

In [9]:
# 获取数据库列名。
def get_table_columns(cur,table_name):
    try:
        # 执行 SQL 查询以获取表的列名
        cur.execute(f"""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_name = '{table_name}';
        """)
        # 获取查询结果
        cols = [desc[0] for desc in cur.fetchall()]
        columns = []
        for col in cols:
            # column = "'"+col+"'"
            columns.append(col)
    except Exception as e:
        print(f"An error occurred: {e}")
    return columns

In [10]:
# 获取数据库表结构，表名对应列名
schema = dict()

table_names = get_table_name(cur)
for name in table_names:
    schema[name] = get_table_columns(cur,name)

In [11]:
def get_column_types(cur, table_name):
    # 执行 SQL 查询以获取表的列名和数据类型
    cur.execute("""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s;
    """, (table_name,))

    # 获取查询结果
    results = cur.fetchall()
    return results

In [12]:
col = get_column_types(cur,"tb_process")
print(col)

[('share', 'integer'), ('deleted', 'smallint'), ('update_time', 'timestamp without time zone'), ('create_time', 'timestamp without time zone'), ('create_user', 'bigint'), ('reference_process_id', 'bigint'), ('parent_id', 'bigint'), ('node_type', 'smallint'), ('extend', 'json'), ('id', 'bigint'), ('version', 'bigint'), ('last_change', 'bigint'), ('category_id', 'bigint'), ('infrastructure_process', 'smallint'), ('quantitative_reference_id', 'bigint'), ('location_id', 'bigint'), ('process_doc_id', 'bigint'), ('dq_system_id', 'bigint'), ('exchange_dq_system_id', 'bigint'), ('social_dq_system_id', 'bigint'), ('last_internal_id', 'integer'), ('currency_id', 'bigint'), ('product_system_id', 'bigint'), ('process_source', 'smallint'), ('node', 'jsonb'), ('ref_id', 'character varying'), ('name', 'character varying'), ('default_allocation_method', 'character varying'), ('synonyms', 'text'), ('dq_entry', 'character varying'), ('tags', 'character varying'), ('library', 'character varying'), ('desc

In [13]:
import re

def getSql(response):
    # Adjust pattern to capture SQL string between #...#
    pattern = r"#(.*?)#"
    sql_list = re.findall(pattern, response, re.DOTALL)  # Use re.DOTALL to allow newline matching
    if sql_list:
        sql_str = sql_list[-1]
        print("SQL:", sql_str)
        return sql_str
    else:
        print("No SQL found")
        return None

In [16]:
# def query_database(text, schema):
#     # 将表结构信息包含在提示中
#     schema_info = "\n".join([f"Table {table}: {', '.join(columns)}" for table, columns in schema.items()])
#     # print(schema_info)
#     prompt= PromptTemplate(
#         input_variables=["schema_info","prompt"],
#         template='''以下是数据库的表结构信息：{schema_info},分析表结构信息.
#         请根据以下描述生成一个符合SQLite数据库语法的SQL查询，并且不能修改给出的数据表列名。
#         描述：{prompt}。
#         要求输出的SQL以#开头,以#结尾，参数类型一律按照字符串处理，样例如下：
#         #SELECT * FROM table#
#         #SELECT COUNT(*) FROM table where Colume_name='abc'#
#         注意不要输出分析过程和其他内容，直接给出SQL语句。
#         '''
#     )
#     chain = LLMChain(llm=llm, prompt=prompt)
#     print(prompt)
#     query = chain.run(
#         {"schema_info":schema_info,"prompt":text}
#     )
#     print(query)
#     sql = getSql(query)
#     # result = execute_query(sql)
#     return sql

In [15]:
def query_database(text, schema):
    # 将表结构信息包含在提示中
    schema_info = "\n".join([f"Table {table}: {', '.join(columns)}" for table, columns in schema.items()])

    # 定义 message 模板
    # messages = [
    #     {
    #         "role": "system",
    #         "content": "你是一个 SQL 生成助手，能够根据用户提供的数据库表结构信息和描述生成符合 SQLite 语法的 SQL 查询。"
    #     },
    #     {
    #         "role": "user",
    #         "content": f'''以下是数据库的表结构信息：{schema_info}。
    #         请根据以下描述生成一个符合 SQLite 数据库语法的 SQL 查询，并且不能修改给出的数据表列名。
    #         描述：{text}。
    #         要求输出的 SQL 以 # 开头，以 # 结尾，参数类型一律按照字符串处理，样例如下：
    #         #SELECT * FROM table#
    #         #SELECT COUNT(*) FROM table where Column_name='abc'#
    #         注意不要输出分析过程和其他内容，直接给出 SQL 语句。'''
    #     }
    # ]
    messages = [
        {
            "role": "system",
            "content": "你是一个 SQL 生成助手，能够根据用户提供的描述生成符合 SQLite 语法的 SQL 查询。"
        },
        {
            "role": "user",
            "content": f'''
            请根据以下描述生成一个符合 SQLite 数据库语法的 SQL 查询，并且不能修改给出的数据表列名。
            描述：{text}。
            要求输出的 SQL 以 # 开头，以 # 结尾，参数类型一律按照字符串处理，样例如下：
            #SELECT * FROM table#
            #SELECT COUNT(*) FROM table where Column_name='abc'#
            注意不要输出分析过程和其他内容，直接给出 SQL 语句。'''
        }
    ]
    # print(messages)

    # 调用 LLM 生成 SQL
    response = llm.invoke(messages)  
    # print(response)
    query = response.content  # 提取生成的 SQL
    # print("生成的 SQL:", query)

    # 提取 SQL 语句
    sql = getSql(query)
    return sql

In [16]:
import pandas as pd
df = pd.read_excel(r"./data/sql_ques_clear.xlsx")
df.head()

,SQL,Question,table
0,select count( * ) as total from tb_process whe...,数据库tb_process表中有多少未删除的数据？,tb_process
1,select tags from tb_process where product_syst...,在数据库tb_process表中，product_system_id为12983111603...,tb_process
2,"select id,ref_id,name,synonyms,category_id,pro...",在数据库tb_process表中，process_type为‘unit_process’且p...,tb_process
3,"select id,ref_id,name,synonyms,category_id,des...",在数据库tb_process表中，id为1300755365750636544的记录的id、...,tb_process
4,"select id,ref_id,name,synonyms,category_id,des...",在数据库tb_process表中，id为9237375的记录的id、ref_id、name、...,tb_process


In [17]:
ques_test = "数据库tb_process表中有多少未删除的数据？"
tmp_dict = dict()
tmp_dict["tb_process"] = schema["tb_process"]
print(tmp_dict)
res = query_database(ques_test,tmp_dict)
print("res:",res)

{'tb_process': ['share', 'deleted', 'update_time', 'create_time', 'create_user', 'reference_process_id', 'parent_id', 'node_type', 'extend', 'id', 'version', 'last_change', 'category_id', 'infrastructure_process', 'quantitative_reference_id', 'location_id', 'process_doc_id', 'dq_system_id', 'exchange_dq_system_id', 'social_dq_system_id', 'last_internal_id', 'currency_id', 'product_system_id', 'process_source', 'node', 'ref_id', 'name', 'default_allocation_method', 'synonyms', 'dq_entry', 'tags', 'library', 'description', 'process_type']}
SQL: SELECT COUNT(*) FROM tb_process WHERE deleted = '0'
res: SELECT COUNT(*) FROM tb_process WHERE deleted = '0'


In [18]:
import time

# Initialize lists to store results and failed indices
sql_gold = []
sql_pred = []
failed_indices = []

for index, item in df.iterrows():
    try:
        prompt = item['Question']
        table = item['table']
        gold = item['SQL']
        
        # Skip if the table is not in table_names
        if table not in table_names:
            print(f"Skipping table: {table} at index {index}")
            continue
        
        # Skip if 'join' is found in the gold SQL
        if 'join' in gold:
            continue
        
        # Get column types and query the database
        colum_type = get_column_types(cur, table)
        tmp = {table: schema[table]}
        sql = query_database(prompt, tmp)
        
        # Append results to the respective lists
        sql_pred.append(sql)
        sql_gold.append(gold)
        
        # Sleep for a second to avoid too many requests in a short time
        time.sleep(1)

    except Exception as e:
        # Handle the exception, print the error, and record the index
        print(f"Error at index {index}: {e}")
        failed_indices.append(index)

# After the loop, you can inspect the failed indices if needed
print(f"Failed indices: {failed_indices}")

SQL: SELECT COUNT(*) FROM tb_process WHERE deleted = '0'
SQL: SELECT tags FROM tb_process WHERE product_system_id = '1298311160348545024' AND deleted = '0'
SQL: SELECT id, ref_id, name, synonyms, category_id, process_type, default_allocation_method, infrastructure_process, quantitative_reference_id, dq_entry, dq_system_id, exchange_dq_system_id, social_dq_system_id, location_id, process_doc_id, currency_id, version, last_change, tags, library, product_system_id, process_source, reference_process_id, node, parent_id, node_type, deleted, create_time, update_time, create_user, extend, share FROM tb_process WHERE process_type = 'unit_process' AND process_source = '0'
SQL: SELECT id, ref_id, name, synonyms, category_id, description, process_type, default_allocation_method, infrastructure_process, quantitative_reference_id, dq_entry, dq_system_id, exchange_dq_system_id, social_dq_system_id, location_id, process_doc_id, currency_id, version, last_change, tags, library, product_system_id, proc

In [33]:
len(sql_gold)

28

In [20]:
len(failed_indices)

38

In [27]:
results = []
id = 0
for i in range(0,66):
    if i in failed_indices:
        results.append(" ")
    else:
        results.append(sql_gold[id])
        id += 1

In [28]:
def save_txt(save_list,path):
    with open(path,"w",encoding="utf-8") as f:
        for item in save_list:
            f.write(item+"\n")